In [1]:
import numpy as np
from copy import copy
import os


def draw_board(paths_x, paths_y, board, save_name):
    
    import matplotlib.pyplot as plt
    width, height = board.shape

    # create a 8" x 8" board
    fig = plt.figure(figsize=[8,8])
    # fig.patch.set_facecolor((1,1,.8))

    ax = fig.add_subplot(111)

    # draw the grid
    for x in range(30):
        ax.plot([x, x], [0,29], color=(0.5,0.5,0.5,1))
    for y in range(30):
        ax.plot([0, 29], [y,y], color=(0.5,0.5,0.5,1))

    # draw paths
    for p in range(len(paths_x)):

        ph = plt.subplot()
        ph.plot(paths_y[p], paths_x[p], linewidth=5, color='black')

    # draw obstacles
    x_axis = []
    y_axis = []
    nets = dict()
    for x in range(width):
        for y in range(height):
            if board[x, y]!=0:
                x_axis.append(y)
                y_axis.append(x)
                if board[x, y]!=1:
                    nets[(x,y)] = board[x, y]

    ax.scatter(x_axis, y_axis, marker='s', s=250, c='k')

    for xy in nets:
        ax.text(xy[1], xy[0], str(int(nets[xy])-1), fontsize=18, color='w',
                horizontalalignment='center', verticalalignment='center')

    # scale the axis area to fill the whole figure
    ax.set_position([0,0,1,1])

    # get rid of axes and everything (the figure background will show through)
    ax.set_axis_off()

    # scale the plot area conveniently (the board is in 0,0..18,18)
    ax.set_xlim(0,29)
    ax.set_ylim(0,29)
    
    fig.savefig(save_name, bbox_inches='tight')

def paths_to_xys(paths):
    paths_x = []
    paths_y = []
    for p in paths:
        xs = [node[0] for node in p]
        ys = [node[1] for node in p]
        paths_x.append(xs)
        paths_y.append(ys)
    
    return paths_x, paths_y


In [29]:
def read_board(file_name):

    board = np.genfromtxt(file_name, delimiter=',')

    # parse the board and get the starts and ends
    start = {}
    finish = {}
    for i in range(board.shape[0]):
        for j in range(board.shape[1]):
            if abs(board[i,j])>=2:
                if board[i,j]<0:
                    finish[-board[i,j]] = (i,j)
                    board[i,j] = abs(board[i,j])
                else:
                    start[board[i,j]] = (i,j)
    return board, start, finish

# rename board names in one folder
def rename_boards(directory, category):

    import re
    import os
    boards_name = os.listdir(directory)
    boards_ids = []
    for name in boards_name:
        idx = int(re.findall(r'\d+', name)[0])
        boards_ids.append(idx)

    boards_ids = sorted(boards_ids)
    # print(boards_ids)
    for new_i, i in enumerate(boards_ids):
        board_name = os.path.join(directory, "board{}.csv".format(i))
        new_board_name = os.path.join(directory, "board_{}{}.csv".format(category, new_i))
        os.rename(board_name, new_board_name)

# separate boards into two categories: A* can connect and A* can't connect
from astar import Astar
import os

boards_II_folder = "/home/ybiao/workspace/RL_routing/boards_II"
if not os.path.isdir(boards_II_folder):
    os.mkdir(boards_II_folder)

board_folder = '/home/ybiao/workspace/RL_routing/boards_I'
for board_idx in range(1000):
    board_path = os.path.join(board_folder,"board{}.csv".format(board_idx))
    board_backup, starts, finishes = read_board(board_path)

    # net_idx = 2
    # for net_idx in range(4, int(max(starts))+1):
    for net_idx in range(2, int(max(starts))+1):
        board = np.copy(board_backup)
        board[board==net_idx]=0
        board[board>=1]=1

        start = starts[net_idx]
        end = finishes[net_idx]

        astar = Astar(board)
        result = astar.run(start, end)

        # print(result)
        if result is None:
            print("Board{} cannot be routed by A star".format(board_idx))
            os.rename(board_path, os.path.join(boards_II_folder, 'board{}.csv'.format(board_idx)))
            break
        else:
            # paths_x, paths_y = paths_to_xys([result])
            for node in result:
                board_backup[tuple(node)]=1
            # draw_board(paths_x, paths_y, board_backup, "test.jpg")

# rename board names for each category
rename_boards(board_folder, 'I')
rename_boards(boards_II_folder, "II")


In [ ]:
# generate supervised learning/burn-in data using A star
